# Prototyping and Testing the execution of the verifiers on the dataset

#### Some config

In [ ]:
NUM_PROC = 1

## Execution Code

In [ ]:
import sys
from datasets import load_dataset,Dataset
from loguru import logger

logger.configure(handlers=[{"sink": sys.stdout, "level": "WARNING"}])
ds = load_dataset("CharlyR/VeriTikz", "raw", split="train")

In [ ]:
from vif.baselines.models import VerEvaluation
from vif.baselines.verifiers_baseline.ver_baseline import TexVerBaseline
from datasets.formatting.formatting import LazyBatch
from PIL import Image


from vif.utils.renderer.tex_renderer import TexRenderer

renderer = TexRenderer()


def execute_verifier_on_dataset(
    verifier: TexVerBaseline, dataset: Dataset, ratio: float = 100
):
    def exec_verif(row):
        metadata = verifier.get_config_metadata()
        ver_eval_input: VerEvaluation = VerEvaluation(
            approach_name=metadata["name"],
            config_metadata=metadata,
            initial_code=row["code"],
            initial_image=row["original_image"],
            initial_instruction=row["instruction"],
            initial_solution=row["solution"],
            initial_solution_image=row["solution_image"],
            expected=row["expected"]
        )
        result = verifier.assess_customization(ver_eval_input)
        result_row = result.model_dump()
        result_row["id"] = row["id"]
        return result_row

    return dataset.map(exec_verif, num_proc=NUM_PROC)